# Change Sizes
Notebook to test how to change the input and output dimentions of an aredy existing system.
This should be done without a totoal recomputation

For now we will ignore the impact on the states and the rank....

We will also only describe a causal system, and all elements not contained in the original matrix are considerd 0 
(Need to change this aferwards, as this could set all elemnts to)

In [ ]:
from tvsclib.strict_system import StrictSystem
from tvsclib.stage import Stage
from tvsclib.system_identification_svd import SystemIdentificationSVD
from tvsclib.toeplitz_operator import ToeplitzOperator
from tvsclib.mixed_system import MixedSystem
import numpy as np
import matplotlib.pyplot as plt
import scipy.linalg as linalg

## Some code to test code

Show system displays the represented matrix and the boundaries.

Check dims checks if the matrix sizes for A,B,C and D fit together

In [ ]:
def show_system(system):
    #function that uses matshow to display the resulting matrix and also shows the divisions
    mat = system.to_matrix()
    plt.matshow(mat)
    x=-0.5
    y=-0.5
    for st in system.stages:
        x+=st.dim_in
        plt.hlines(y,-0.5,x)
        plt.vlines(x,y,mat.shape[0]-0.5)
        y+=st.dim_out
                   
        

def check_dims(system,dim_state_in=0,dim_state_out=0,text_output=True,return_report=False):
    rep = ""
    correct = True
    dim_state = dim_state_in
    for i,st in enumerate(system.stages):
        #check if the state input is correct for A and C
        if st.A_matrix.shape[1] != dim_state:
            correct = False
            rep = rep + "Problem at index "+str(i)+": State dims of A do not match: old:"+str(dim_state)+ \
                  " new: "+str(st.A_matrix.shape[1])+"\n"
        if st.C_matrix.shape[1] != dim_state:
            correct = False
            rep = rep + "Problem at index "+str(i)+": State dims of C do not match: old:"+str(dim_state)+ \
                  " new: "+str(st.C_matrix.shape[1])+"\n"
            
        #check if the state output of A and B match
        dim_state = st.A_matrix.shape[0]
        if st.B_matrix.shape[0] != dim_state:
            correct = False
            rep = rep + "Problem at index "+str(i)+": State dims of A and B do not match: A:"+str(dim_state)+ \
                  "B: "+str(st.B_matrix.shape[0]) + "\n"
            
        #check if the input dims match
        if st.B_matrix.shape[1] != st.D_matrix.shape[1]:
            correct = False
            rep = rep + "Problem at index "+str(i)+": Input dims of B and D do not match: B:"+str(st.B_matrix.shape[1])+ \
                  "D: "+str(st.D_matrix.shape[1]) +"\n"
        
        #check if the output states match
        if st.C_matrix.shape[0] != st.D_matrix.shape[0]:
            correct = False
            rep = rep + "Problem at index "+str(i)+": Output dims of C and D do not match: C:"+str(st.C_matrix.shape[0])+ \
                  "D: "+str(st.D_matrix.shape[0]) +"\n"
    if dim_state != dim_state_out:
        correct = False
        rep = rep + "final state dim does not match"
    if text_output:
        if correct:
            print("Matrix shapes are correct")
        else:
            print("Matrix shapes are not correct")
            print(rep)
    if return_report:
        return correct,rep
    else:
        return correct


In [ ]:
#check_dims(system,return_report=True,text_output=False)
#show_system(system)

In [ ]:
#system.reachability_matricies()

# Move to left

Moving one bopundary to the left

This requires the following:
- Cut one collumn from $D_v$
- Add the right collumn form $C_{v+1}B_v$ to $D_{v+1}$ 
- Cut one column from $B_v$
- Add the rightmoust collumn of $A_{v+1}B_v$ to $B_{v+1}$ (basically just make sure that it will result in the same state after the timestep $v+1$)

In [ ]:
def move_left(sys,v):
    #function to move a boundary left 
    #v is the index to the left of the boundary
    
    #set the appropirate D_v
    sys.stages[v].D_matrix=sys.stages[v].D_matrix[:,:-1]
    #extract last collumn of B_v
    col = sys.stages[v].B_matrix[:,-1].reshape((-1,1))
    sys.stages[v].B_matrix = sys.stages[v].B_matrix[:,:-1]
    #set the appropriate D_v+1
    sys.stages[v+1].D_matrix=np.hstack((sys.stages[v+1].C_matrix@col,sys.stages[v+1].D_matrix))
    sys.stages[v+1].B_matrix=np.hstack((sys.stages[v+1].A_matrix@col,sys.stages[v+1].B_matrix))
    
    #make it minimal if nececarry
    U,s,Vt= np.linalg.svd(np.hstack((sys.stages[v].A_matrix,sys.stages[v].B_matrix)),full_matrices=False)
    display(s)
    tol = 1e-13

    if abs(s[-1])<tol:
        #not full rank-> not minimal -> reduce dimentions
        print('remove')
        sys.stages[v].A_matrix = Vt[:-1,:sys.stages[v].A_matrix.shape[1]]*s[:-1]#TODO: check these multiplications
        sys.stages[v].B_matrix = Vt[:-1,sys.stages[v].A_matrix.shape[1]:]*s[:-1]
        
        sys.stages[v+1].A_matrix=sys.stages[v+1].A_matrix@U[:,:-1]
        sys.stages[v+1].C_matrix=sys.stages[v+1].C_matrix@U[:,:-1]

# Move down

Moving one bopundary to the left

This requires the following:
- Cut one row from $D_{v+1}$
- Add the upper row form $C_{v+1}B_{v}$ to $D_{v}$ 
- Cut upper row from $C_{v+1}$
- Add the uppermost collumn of $C_{v+1}A_v$ to $C_{v}$ 

In [ ]:
def move_down(sys,v):
    #function to move a boundary down
    #v is the index above of the boundary
    
    #cot row of D_v+1
    sys.stages[v+1].D_matrix=sys.stages[v+1].D_matrix[1:,:]
    #extract first row of C_v+1
    row = sys.stages[v+1].C_matrix[0,:].reshape((1,-1))
    sys.stages[v+1].C_matrix = sys.stages[v+1].C_matrix[1:,:]
    #set the appropriate D_v and C_v+1
    sys.stages[v].D_matrix=np.vstack((sys.stages[v].D_matrix,row@sys.stages[v].B_matrix))
    sys.stages[v].C_matrix=np.vstack((sys.stages[v].C_matrix,row@sys.stages[v].A_matrix))
    
    #make it minimal if nececarry
    U,s,Vt= np.linalg.svd(np.vstack((sys.stages[v+1].A_matrix,sys.stages[v+1].C_matrix)),full_matrices=False)
    display(U)
    display(s)
    display(Vt)
    tol = 1e-13

    if abs(s[-1])<tol:
        #not full rank-> not minimal -> reduce dimentions
        print('remove')
        sys.stages[v+1].A_matrix = U[:sys.stages[v+1].A_matrix.shape[0],:-1]*s[:-1].reshape(-1,1)
        sys.stages[v+1].C_matrix = U[sys.stages[v+1].A_matrix.shape[0]:,:-1]*s[:-1].reshape(-1,1)
        
        sys.stages[v].A_matrix=Vt[:-1,:]@sys.stages[v].A_matrix
        sys.stages[v].B_matrix=Vt[:-1,:]@sys.stages[v].B_matrix
        

In [ ]:
def move_right(sys,v,d_new=None):
    #function to move a boundary left 
    #v is the index to the left of the boundary
    
    #first collumn from B and D
    b = sys.stages[v+1].B_matrix[:,0].reshape((-1,1))
    d = sys.stages[v+1].D_matrix[:,0].reshape((-1,1))
    
    sys.stages[v+1].B_matrix=sys.stages[v+1].B_matrix[:,1:]
    sys.stages[v+1].D_matrix=sys.stages[v+1].D_matrix[:,1:]
    
    #check if [d;b] in range(C_v+1;Av+1)
    U,s,Vt= np.linalg.svd(np.vstack((sys.stages[v+1].C_matrix,sys.stages[v+1].A_matrix)),full_matrices=True)
    eps = 1e-13
    r = np.count_nonzero(s>=eps)
    a = U.T@np.vstack((d,b))
    print("r="+str(r))
    display(a)
    if np.any(np.abs(a[r:])>eps):
        #not in range -> add a ned dimention to the state
        sys.stages[v].B_matrix = np.block([
            [sys.stages[v].B_matrix,np.zeros((sys.stages[v].B_matrix.shape[0],1))],
            [np.zeros((1, sys.stages[v].B_matrix.shape[1])), 1     ]
            ])
        sys.stages[v].A_matrix = np.vstack((sys.stages[v].A_matrix,np.zeros((1,sys.stages[v].A_matrix.shape[1]))))
        sys.stages[v+1].A_matrix = np.hstack((sys.stages[v+1].A_matrix,b))
        sys.stages[v+1].C_matrix = np.hstack((sys.stages[v+1].C_matrix,d))
        
    else:
        #in range -> no need for an additional dim
        m = Vt.T[:,:r]@(a[:r].flatten()/s[:r])
        print("m:")
        display(m)
        sys.stages[v].B_matrix = np.hstack((sys.stages[v].B_matrix,m.reshape((-1,1))))
    
    #set the appropirate D_v
    if not d_new:
        d_new = np.zeros((sys.stages[v].D_matrix.shape[0],1))
    sys.stages[v].D_matrix=np.hstack((sys.stages[v].D_matrix,d_new))
    

In [ ]:
def move_up(sys,v,d_new=None):
    #function to move a boundary up
    #v is the index above of the boundary
    
    #last row from C and D
    c = sys.stages[v].C_matrix[-1,:].reshape((1,-1))
    d = sys.stages[v].D_matrix[-1,:].reshape((1,-1))
    
    sys.stages[v].C_matrix=sys.stages[v].C_matrix[:-1,:]
    sys.stages[v].D_matrix=sys.stages[v].D_matrix[:-1,:]
    
    #check if [d;c]^T in range(B_v;A_v)^T
    U,s,Vt= np.linalg.svd(np.vstack((sys.stages[v].B_matrix.T,sys.stages[v].A_matrix.T)),full_matrices=True)
    eps = 1e-13
    r = np.count_nonzero(s>=eps)
    a = U.T@np.vstack((d.T,c.T))
    print("r="+str(r))
    display(a)
    if np.any(np.abs(a[r:])>eps):
        #not in range -> add a ned dimention to the state
        sys.stages[v+1].C_matrix = np.block([
            [np.zeros((1, sys.stages[v+1].C_matrix.shape[1])), 1     ],
            [sys.stages[v+1].C_matrix,np.zeros((sys.stages[v+1].C_matrix.shape[0],1))]
            ])
        sys.stages[v+1].A_matrix = np.hstack((sys.stages[v+1].A_matrix,np.zeros((sys.stages[v+1].A_matrix.shape[0],1))))
        sys.stages[v].A_matrix = np.vstack((sys.stages[v].A_matrix,c))
        sys.stages[v].B_matrix = np.vstack((sys.stages[v].B_matrix,d))
        
    else:
        #in range -> no need for an additional dim
        m = Vt.T[:,:r]@(a[:r].flatten()/s[:r])
        print("m:")
        display(m)
        sys.stages[v+1].C_matrix = np.vstack((m.reshape((1,-1)),sys.stages[v+1].C_matrix))
    
    #set the appropirate D_v
    if not d_new:
        d_new = np.zeros((1,sys.stages[v+1].D_matrix.shape[1]))
    sys.stages[v+1].D_matrix=np.vstack((d_new,sys.stages[v+1].D_matrix))
    

## Test the mooves

## Test them if state dims are constant

The represented matrix is Rank 1. This means that the state is always 1-dim.

In [ ]:
matrix = np.arange(0,12).reshape((-1,1))@np.arange(0,12).reshape((1,-1))
dims_in =  np.array([2, 1, 2, 1])*2
dims_out = np.array([1, 2, 1, 2])*2
T = ToeplitzOperator(matrix, dims_in, dims_out)
S = SystemIdentificationSVD(T,epsilon=1e-12)

system = MixedSystem(S).causal_system
matrix_ref = system.to_matrix()
show_system(system)

In [ ]:
move_right(system,0)
check_dims(system)
display(system.dims_state)
show_system(system)

In [ ]:
move_left(system,0)
check_dims(system)
display(system.dims_state)
show_system(system)

print("Diff:",np.max(np.abs(system.to_matrix()-matrix_ref)))

In [ ]:
move_up(system,2)
check_dims(system)
display(system.dims_state)
show_system(system)

In [ ]:
move_down(system,2)
check_dims(system)
display(system.dims_state)
show_system(system)

print("Diff:",np.max(np.abs(system.to_matrix()-matrix_ref)))

## Check the shifts if the number of states changes

In [ ]:
A = np.random.rand(12,12)
Q,R = np.linalg.qr(A)


matrix = np.zeros((12,12))
matrix[:,:]=(Q[:,:4]@np.array([1,1,0,0])).reshape(-1,1)
matrix[:,4:6]=(Q[:,:4]@np.array([1,1,0.1,0])).reshape(-1,1)
#plt.imshow(matrix)


dims_in =  np.array([2, 1, 2, 1])*2
dims_out = np.array([1, 2, 1, 2])*2
T = ToeplitzOperator(matrix, dims_in, dims_out)
S = SystemIdentificationSVD(T,epsilon=1e-12)
system = MixedSystem(S).causal_system

matrix_ref = system.to_matrix()
show_system(system)
system.dims_state

In [ ]:
move_right(system,0)
check_dims(system)
display(system.dims_state)
show_system(system)

In [ ]:
move_left(system,0)
check_dims(system)
display(system.dims_state)
show_system(system)
print("Diff:",np.max(np.abs(system.to_matrix()-matrix_ref)))

In [ ]:
A = np.random.rand(12,12)
Q,R = np.linalg.qr(A)


matrix = np.zeros((12,12))
matrix[:,:]=(Q[:,:4]@np.array([1,1,0,0])).reshape(-1,1)
matrix[:,4:6]=(Q[:,:4]@np.array([1,1,0.1,0])).reshape(-1,1)
#plt.imshow(matrix)
matrix = matrix.T

dims_in =  np.array([2, 1, 2, 1])*2
dims_out = np.array([1, 2, 1, 2])*2
T = ToeplitzOperator(matrix, dims_in, dims_out)
S = SystemIdentificationSVD(T,epsilon=1e-12)
system = MixedSystem(S).causal_system

matrix_ref = system.to_matrix()
show_system(system)
system.dims_state

In [ ]:
move_up(system,1)
check_dims(system)
display(system.dims_state)
show_system(system)

In [ ]:
move_down(system,1)
check_dims(system)
display(system.dims_state)
show_system(system)
print("Diff:",np.max(np.abs(system.to_matrix()-matrix_ref)))

# Combine and Split

In [ ]:
def combine(sys,v,D_new = 0):
    #function that combnines the timestep v with the following timestep
    if not D_new:
        D_new=np.zeros((sys.stages[v].D_matrix.shape[0],sys.stages[v+1].D_matrix.shape[1]))
    sys.stages[v].D_matrix= np.block([
            [sys.stages[v].D_matrix                         , D_new    ],
            [sys.stages[v+1].C_matrix@sys.stages[v].B_matrix,  sys.stages[v+1].D_matrix]
            ])
    sys.stages[v].B_matrix=np.hstack((sys.stages[v+1].A_matrix@sys.stages[v].B_matrix,sys.stages[v+1].B_matrix))
    sys.stages[v].C_matrix=np.vstack((sys.stages[v].C_matrix,sys.stages[v+1].C_matrix@sys.stages[v].A_matrix))
    sys.stages[v].A_matrix = sys.stages[v+1].A_matrix@sys.stages[v].A_matrix
    
    del sys.stages[v+1]
    

In [ ]:
def split(sys,v,indices=(-1,-1),tol = 5e-15):
    #function that splits the timestep v in two timesteps
    #the parameter indices determine how to split the output and input
    #indices[0]: last row in the first step
    #indices[1]: last collumn in the first timestep
    
    if indices[0]<0:
        indices[0] = np.floor(sys.stages[v].D_matrix.shape[0]/2)
    if indices[1]<0:
        indices[1] = np.floor(sys.stages[v].D_matrix.shape[1]/2)        
    
    U,s,Vt= np.linalg.svd(np.block([
            [sys.stages[v].A_matrix,sys.stages[v].B_matrix[:,:indices[1]]],
            [sys.stages[v].C_matrix[indices[0]:,:],sys.stages[v].D_matrix[indices[0]:,:indices[1]]]
            ]))
    
    n_in = sys.stages[v].A_matrix.shape[1] #dims of state bevore and after
    n_out = sys.stages[v].A_matrix.shape[0]
    display(s)
    n = np.count_nonzero(s>tol)
    print("n:",n)
    
    rs = np.sqrt(s)
    Us=U*rs
    sVt=Vt*rs.reshape(-1,1)
    stage_a=Stage(sVt[:n,:n_in],
                  sVt[:n,n_in:],
                  sys.stages[v].C_matrix[:indices[0],:],
                  sys.stages[v].D_matrix[:indices[0],:indices[1]])
    stage_b=Stage(Us[:n_out,:n],
                  sys.stages[v].B_matrix[:,indices[1]:],
                  Us[n_out:,:n],
                  sys.stages[v].D_matrix[indices[0]:,indices[1]:])
    
    sys.stages.insert(v,stage_a)
    sys.stages[v+1]=stage_b
    

In [ ]:
matrix = np.array([
        [5,     4,     6,     1,     4,     2],
        [2,     3,     2,     1,     3,     4],
        [6,     3,     5,     4,     1,     1],
        [3,     5,     5,     5,     3,     4],
        [2,     4,     3,     6,     1,     2],
        [2,     4,     4,     1,     5,     4]
])
matrix = np.vstack((np.hstack((matrix,matrix)),np.hstack((matrix,matrix))))

dims_in =  np.array([2, 1, 2, 1])*2
dims_out = np.array([1, 2, 1, 2])*2
T = ToeplitzOperator(matrix, dims_in, dims_out)
S = SystemIdentificationSVD(T,epsilon=1e-12)

system = MixedSystem(S).causal_system


In [ ]:
show_system(system)

In [ ]:
combine(system,1)
check_dims(system)
display(system.dims_state)
show_system(system)

In [ ]:
split(system,1,indices=(4,2))
check_dims(system)
display(system.dims_state)
show_system(system)

Make a rank1 matrix and add an off point to make the rank change, if it is included.

In [ ]:
matrix = np.arange(0,12).reshape((-1,1))@np.arange(0,12).reshape((1,-1))
matrix[6,5]=10
dims_in =  np.array([2, 1, 2, 1])*2
dims_out = np.array([1, 2, 1, 2])*2
T = ToeplitzOperator(matrix, dims_in, dims_out)
S = SystemIdentificationSVD(T,epsilon=1e-12)

system = MixedSystem(S).causal_system
show_system(system)

matrix_ref = system.to_matrix()
display(system.dims_state)

In [ ]:
combine(system,1)
check_dims(system)
display(system.dims_state)
show_system(system)

In [ ]:
split(system,1,indices=(4,2))
check_dims(system)
display(system.dims_state)
show_system(system)

print("Diff:",np.max(np.abs(system.to_matrix()-matrix_ref)))

# Test check dims

In [ ]:
system.stages[0].A_matrix=np.eye(3)

In [ ]:
check_dims(system)

In [ ]:
system.stages[2].D_matrix=np.eye(3)
check_dims(system)

In [ ]:
check_dims(system)